In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from Loader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [2]:
BATCH_SIZE = 128

# x_train = None
# x_val = None
x_train = DataLoader(BATCH_SIZE, 'training_generator')
x_val = DataLoader(BATCH_SIZE, 'training_generator', val=True)

In [3]:
lossHistory = []
valHistory = []

class NnueCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    lossHistory[-1].append(logs['loss'])
    if batch and batch % 15000 == 0:

      pred = self.model.predict(x_val)
      loss = (pred.flatten() - x_val.labels.flatten()[0:len(x_val)*BATCH_SIZE]) ** 2
      loss = np.sum(loss) / len(loss)
      valHistory[-1].append(loss)

      for i in lossHistory:
        plt.plot(i[20:], linewidth=0.5)
        plt.savefig("loss.png", dpi=400)
      plt.clf()

      for i in valHistory:
        plt.plot(i, linewidth=0.5)
        plt.savefig("val.png", dpi=400)
      
      plt.clf()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [4]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=4)

model = Sequential()
model.add(Dense(128, input_shape=(769,), activation=clipped_relu))
model.add(Dense(32, activation=clipped_relu))
model.add(Dense(32, activation=clipped_relu))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

2022-09-30 22:54:58.876949: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 22:54:58.881249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 22:54:58.881621: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 22:54:58.882269: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
model = tf.keras.models.load_model("production/")

In [7]:
lossHistory.append([])
valHistory.append([])
model.fit(x_train, validation_data=x_val, epochs=100, callbacks=[NnueCallbacks(), EarlyStopping(patience=1)])

#Epoch 2/2
#10906/10906 [==============================] - 225s 21ms/step - loss: 0.0243 - mae: 0.0866

Epoch 1/100


: 

: 

In [7]:
import Features
import numpy as np

FEN = "r3n1k1/2qbbppp/4p3/3p2P1/2p2P2/2N1P3/Pr1B2BP/R2Q1RK1 b - - 2 19"
features = Features.get(FEN)
model.predict(np.array([features]))

array([[0.15982483]], dtype=float32)

In [6]:
model.save("production")

2022-09-15 17:51:59.322128: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: production/assets
